# CST 8502 FINAL PROJECT

By:  
Charles-Antoine Campeau  
Joshua Ayyasamy  
Mubarak husain Shaikh  
Curtis Sloan 

Submitted to Dr. Anu Thomas in partial fulfillment of the requirements of CST 8502 

Algonquin College Artificial Intelligence Software Development

2023-11-26

In [ ]:
import pandas as pd
import numpy as np
import math
import re
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import seaborn as sns


## DATA PREPARATION

In [ ]:
# Import the CSV
bicycleTheftData = pd.read_csv("bicycle-thefts - 4326.csv")

In [ ]:
# Remove the unnecessary attributes
bicycleTheftData = bicycleTheftData.drop(columns=["_id","EVENT_UNIQUE_ID", "OCC_DATE", "OCC_YEAR", "OCC_DAY",
                                                   "OCC_DOY", "REPORT_DATE", "REPORT_YEAR", "REPORT_MONTH", "REPORT_DOW", "REPORT_DAY", 
                                                     "REPORT_DOY", "REPORT_HOUR", "LOCATION_TYPE", "BIKE_MODEL", "STATUS", "geometry"])

## DATA CLEANING
The data cleaning was separated between all members.  Everyone's contribution is indicated

### Charles

In [ ]:
# Import the CSV with the BIKE_MAKE changes
makesData = pd.read_csv("MAKESREPLACEMENT.CSV", delimiter=";")
# Set the index as the good values and convert the strings to list
makesData.set_index("NEW", inplace=True)
makesData["OLD"] = makesData["OLD"].str.split(",")

# Convert the list to a series
makesSeries = makesData["OLD"].explode()

In [ ]:
def GetMake(make):
    """Correct wrongfully inputted bicycle makes \n
    Parameter:
    ------------
    make: The instance bicycle make

    Return:
    -----------
    The properly written make
    """
    
    try:
        index = makesSeries.index[makesSeries.str.contains(re.escape(make))][0]
        return index
    except: 
        return make

In [ ]:
# Fix the errors in the BIKE_MAKE attribute
bicycleTheftData["BIKE_MAKE"] = bicycleTheftData["BIKE_MAKE"].apply(GetMake)

### Joshua

In [ ]:
bicycleTheftData['BIKE_SPEED'] = bicycleTheftData['BIKE_SPEED'].fillna(bicycleTheftData['BIKE_SPEED'].mean())

In [ ]:
low = bicycleTheftData['BIKE_COST'].quantile(.25)
average = bicycleTheftData['BIKE_COST'].quantile(.5)
high = bicycleTheftData['BIKE_COST'].quantile(.75)
bicycleTheftData['BIKE_COST_CATEGORY'] = np.select(
    [
        bicycleTheftData['BIKE_COST'].isna(),
        bicycleTheftData['BIKE_COST'] <= low,
        (bicycleTheftData['BIKE_COST'] > low) & (bicycleTheftData['BIKE_COST'] <= average),
        (bicycleTheftData['BIKE_COST'] > average) & (bicycleTheftData['BIKE_COST'] <= high),
        bicycleTheftData['BIKE_COST'] > high
    ],
    [
        'NK',
        'Low',
        'Average',
        'High',
        'Luxury'
    ],
    default='Unknown'
)

In [ ]:
bicycleTheftData = bicycleTheftData.drop(['BIKE_COST'], axis =1)